In [28]:
import warnings
warnings.filterwarnings(action='ignore')


import pandas as pd

In [29]:
test_y_df = pd.read_csv('./datasets/titanic_gender.csv', encoding='utf-8')
test_df = pd.read_csv('./datasets/titanic_test.csv', encoding='utf-8')
train_df =pd.read_csv('./datasets/titanic_train.csv', encoding='utf-8')



In [30]:
train_df.head()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [31]:
train = train_df[['Pclass','Sex', 'Age', 'Survived']]
test = test_df[['Pclass','Sex', 'Age']]
test_y = test_y_df['Survived']

In [32]:

train = train.dropna(axis=0)
train.isnull().sum()

Pclass      0
Sex         0
Age         0
Survived    0
dtype: int64

In [33]:
train.describe()
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    714 non-null    int64  
 1   Sex       714 non-null    object 
 2   Age       714 non-null    float64
 3   Survived  714 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 27.9+ KB


In [34]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data = encoder.fit_transform(train['Sex'])


In [35]:
train['Sex'] = data

In [36]:
train.head()

,Pclass,Sex,Age,Survived
0,3,1,22.0,0
1,1,0,38.0,1
2,3,0,26.0,1
3,1,0,35.0,1
4,3,1,35.0,0


In [49]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train[['Age']])
data = scaler.transform(train[['Age']])
train['Age'] = data
train.head()

,Pclass,Sex,Age,Survived
0,3,1,0.271174,0
1,1,0,0.472229,1
2,3,0,0.321438,1
3,1,0,0.434531,1
4,3,1,0.434531,0


In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [59]:
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
lr = LogisticRegression()

X =train.drop('Survived', axis=1)
y =train['Survived']

result_dict = {
    'dt':[],
    'rf':[],
    'lr':[]
}

for index, algorithm in enumerate([dt, rf, lr]):
    result = cross_val_score(algorithm, X,y)
    if index == 0 :
        result_dict['dt'] = result
    elif index == 1:
        result_dict['rf'] = result
    else:
        result_dict['lr'] = result

In [60]:
result_dict


{'dt': array([0.77622378, 0.81818182, 0.8041958 , 0.81118881, 0.83098592]),
 'rf': array([0.76223776, 0.8041958 , 0.83216783, 0.83216783, 0.82394366]),
 'lr': array([0.74825175, 0.83916084, 0.77622378, 0.74825175, 0.80985915])}

In [62]:
# 한길 방법 
result = {}
for i, j in enumerate([dt, rf, lr]) :
  result[i] = cross_val_score(j, X, y)

In [63]:
import numpy as np
print(np.mean(result_dict['dt']))
print(np.mean(result_dict['rf']))
print(np.mean(result_dict['lr']))

0.8081552250566336
0.8109425785482124
0.7843494533635379


In [65]:
test
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Sex     418 non-null    object 
 2   Age     332 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.9+ KB


In [67]:
test['Survived'] = test_y_df['Survived']
test

,Pclass,Sex,Age,Survived
0,3,male,34.5,0
1,3,female,47.0,1
2,2,male,62.0,0
3,3,male,27.0,0
4,3,female,22.0,1
...,...,...,...,...
413,3,male,NaN,0
414,1,female,39.0,1
415,3,male,38.5,0
416,3,male,NaN,0


In [68]:
test.dropna(axis=0, inplace=True)

In [70]:
test.isna().sum()

Pclass      0
Sex         0
Age         0
Survived    0
dtype: int64

In [79]:
encoder = LabelEncoder()
data = encoder.fit_transform(test[['Sex']])
test['Sex']=data

In [80]:
scaler = MinMaxScaler()
scaler.fit(test[['Age']])
data = scaler.transform(test[['Age']])
test['Age'] = data
test.head()


,Pclass,Sex,Age,Survived
0,3,1,0.452723,0
1,3,0,0.617566,1
2,2,1,0.815377,0
3,3,1,0.353818,0
4,3,0,0.287881,1


In [92]:
X_test = test.drop('Survived', axis =1)


In [93]:
from sklearn.metrics import accuracy_score

In [95]:

rf.fit(X, y)
pred_rf = rf.predict(X_test)

accuracy_score(test['Survived'], pred_rf)

0.8403614457831325

In [96]:

dt.fit(X, y)
pred_dt = dt.predict(X_test)

accuracy_score(test['Survived'], pred_dt)

0.8192771084337349

In [97]:

lr.fit(X, y)
pred_lr = lr.predict(X_test)

accuracy_score(test['Survived'], pred_lr)

0.927710843373494